In [1]:
#针对人工review完的数据，处理未被分类的单类商品和人工摘出来的错误商品和没有sku的商品
import numpy as np
import time

def load_Glove_data():
    item={}
    with open('./Glove/vec_glv_475_rmcid3.txt', encoding='utf8') as file:
        for line in file:
            try:
                vec=[]
                pws=[pw_idx for pw_idx in line.strip().split(" ")]
                id=pws[0]
                for content in pws[1:]:
                    vec.append(float(content))
                item[id]=np.array(vec).T
            except:
                continue
    return item
def _func(vec1,vec2):
    #vec1 = vec_Glove[i]
    #vec2 = vec_Glove[j]
    val1 = np.sqrt(sum([item ** 2 for item in vec1]))
    val2 = np.sqrt(sum([item ** 2 for item in vec2]))
    sim = vec1.dot(vec2.T) / (val1 * val2)
    return sim
def load_data_relevant():
    edges={}
    with open('base_relevant_-1rmcid3_all', encoding='utf8') as file:
        for line in file:
            item=line.split(":")
            key=item[0]
            cons = [pw for pw in item[1].strip().split(",") if pw!='']
            edges[key]=cons
    return edges
def rn_num():
    rn_num={}
    code_pw={}
    for line in open('./id1_pw_num', encoding='utf8'):
        cons = line.strip().split("\t")
        try:
            rn_num[cons[0]]=cons[2]
            code_pw[cons[2]]=cons[0]
        except:
            print(cons[0])
    return rn_num,code_pw
def name_id(name):
    return rn_num[name]

vec_Glove=load_Glove_data()
print("load_Glove_data done")
#rn序号对应的cid1-pw
rn_num,code_pw=rn_num()
edges=load_data_relevant()
print('相关数据表读取完毕')

load_Glove_data done
相关数据表读取完毕


In [4]:
#----------------------人工review前的数据------------------------------------
#读取根据阈值0.3处理的过的单类商品，如果在人工review阶段被‘-’掉，则不能再加入该大类
'''def Data_processed_by_thre():
    data_processed_by_thre=[]
    with open('manual_judge_word_done_0.3', encoding='utf8') as file:
        for line in file:
            cons = [pw for pw in line.strip().split(",") if pw!='']
            data_processed_by_thre.append(cons)
    return data_processed_by_thre'''

#读取未进入人工review阶段的单类商品,对他们进行阈值0.25调整后划分，剩下的单独成类
def Data_without_processed_by_thre():
    data_without_processed_by_thre=[]
    with open('manual_judge_word_rm_0.3', encoding='utf8') as file:
        for line in file:
            cons = line.strip()
            data_without_processed_by_thre.append(cons)
    return data_without_processed_by_thre

#----------------------人工review后的数据------------------------------------
#读取人工review完毕后的总数据
def load_review_data():
    #分为分好类的数据，删选出来不属于该类的商品和没有sku的商品
    data_done=[]
    data_del=[]
    data_del_cluster={}
    data_without_sku=[]
    with open('data_review', encoding='utf8') as file:
        for line in file:
            cons = [pw.strip() for pw in line.strip().split(",") if pw!='']
            #print(cons[0][0])
            if cons[0][0]=='-':
                for item in cons:
                    data_del.append(item.strip('-').strip())
            elif cons[0][0]=='=':
                for item in cons:
                    data_without_sku.append(item.strip('=').strip())
            else:
                data_done.append(cons)
    #print(data_done,data_del,data_without_sku)
    return data_done,data_del,data_without_sku
              
              
#已经被按0.3处理过的词[]
#data_processed_by_thre=Data_processed_by_thre()
#按照0.3仍未被处理的词，改用0.25阈值
data_without_processed_by_thre=Data_without_processed_by_thre()
#人工review完后，大类数据、删除数据和无sku数据
data_done,data_del,data_without_sku=load_review_data()

#----------------------首先针对无sku且排序>260000------------------------------------ 
#这部分处理完后，按照阈值0.4放入，否则认为找不到对应sku，单独成类
def rm_pw_without_sku(data_without_sku):
    data_without_sku_new=[]
    rm_sku_num=0
    for pw in data_without_sku:
        if pw=='':
            continue
        if int(rn_num[pw])>=260000:
            rm_sku_num+=1
            continue
        else:
            data_without_sku_new.append(pw)
    print('The number of remove pw without sku is:',rm_sku_num)
    return data_without_sku_new
data_without_sku=rm_pw_without_sku(data_without_sku)


#----------------------定义合并数据的方法------------------------------------ 
#需要被归入的大类数据，需要合并的单类数据，输入阈值
def data_merge(base,process,thre):
    #维护长度为3的堆
    top_n = 3
    base_vec={}
    base_id_code={}
    base_id=0
    #编码所在的list序列号
    code_list={}
    for cons in base:
        vec=np.array([0.0 for i in range(300)])
        tmp_code=[]
        for item in cons:
            if item=='' or item=='=' or item=='-':
                continue
            vec+=vec_Glove[name_id(item)]
            tmp_code.append(name_id(item))
            code_list[name_id(item)]=base_id
        base_id_code[base_id]=tmp_code
        base_vec[base_id]=vec/len(cons)
        base_id+=1
    remove_pw=[]
    line=0
    for pw in process:
        if line%1000==0:
            print(line)
        line+=1
        #vec_tmp为要处理的pw的词向量
        vec_tmp = np.array(vec_Glove[name_id(pw)])
        keys = []
        vals = []
        if name_id(pw) in edges.keys():
            for edge in edges[name_id(pw)]:
                try:
                    vec_edge=vec_Glove[edge]
                    val=_func(vec_tmp,vec_edge)
                    if len(keys) < top_n:
                            keys.append(edge)
                            vals.append(val)
                    else:
                        min_val = vals[0]
                        min_idx = 0
                        for i in range(top_n):
                            if vals[i] < min_val:
                                min_val = vals[i]
                                min_idx = i
                        if val > min_val:
                            keys[min_idx] = edge
                            vals[min_idx] = val
                except:
                    continue
        key_val={}
        for i in range(len(keys)):
            key_val[keys[i]]=vals[i]
        new=sorted(key_val.items(),key=lambda x:x[1],reverse=True)
        keys,vals=[],[]
        for content in new:
            keys.append(content[0])
            vals.append(content[1])
        for i in range(top_n):
            try:
                if keys[i] in code_list.keys():
                    if vals[i]>thre:
                        base[code_list[keys[i]]].append(pw)
                        #满足单个在其他类中，即被处理的词
                        if pw not in remove_pw:
                            remove_pw.append(pw)
                        break
            except:                    
                continue
    process_left=[]
    for item in process:
        if item not in remove_pw:
            process_left.append(item)
    print('Data_merge is done!!')
    return base,process_left,remove_pw


#----------------------处理无sku且排序小于26W的词------------------------------------ 
data_done,data_without_sku_left,data_without_sku_remove_pw=data_merge(data_done,data_without_sku,0.4)
print('无sku数据处理完毕，根据0.4阈值，总的需要处理个数为{}，剩余未处理pw个数为{}，已处理个数为{}'.format(len(data_without_sku_left),len(data_without_sku_left),len(data_without_sku_remove_pw)))

#----------------------处理按照0.3仍未被处理的词，改用0.25阈值------------------------------------ 
data_done,data_without_processed_by_thre_left,data_without_processed_by_thre_remove_pw=data_merge(data_done,data_without_processed_by_thre,0.25)
print('换用0.25阈值的数据处理完毕，总的需要处理个数为{}，剩余未处理pw个数为{}，已处理个数为{}'.format(len(data_without_processed_by_thre),len(data_without_processed_by_thre_left),len(data_without_processed_by_thre_remove_pw)))

#----------------------处理人工review出来不符合类别的词------------------------------------ 
#针对剩余的单类商品，做二轮聚类处理
#data_done,data_del_left,data_del_remove_pw=data_merge(data_done,data_del,0.25)



#----------------------写入------------------------------------ 
#data_done+++data_without_sku_left+++++++data_without_processed_by_thre_left+++++

file = open('data_review_out', 'w', encoding='utf8')
for i in range(len(data_done)):
    for j in range(len(data_done[i])):
        file.write(data_done[i][j])
        file.write(", " if j <len(data_done[i])-1 else "")
    file.write("\n")
file.write("-----------------------------------无sku的没有划分进大类的--------------------------------------------------------------------")
file.write("\n")
for i in range(len(data_without_sku_left)):
    file.write(data_without_sku_left[i])
    file.write("\n")
file.write("------------------------------------阈值小于0.25的单类-------------------------------------------------------------------")
file.write("\n")
for i in range(len(data_without_processed_by_thre_left)):
    file.write(data_without_processed_by_thre_left[i])
    file.write("\n")
file.write("-------------------------------------人工删除的单类------------------------------------------------------------------")
file.write("\n")
for i in range(len(data_del)):
    file.write(data_del[i])
    file.write("\n")
file.close()


The number of remove pw without sku is: 389
0
Data_merge is done!!
无sku数据处理完毕，根据0.4阈值，总的需要处理个数为256，剩余未处理pw个数为256，已处理个数为195
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
Data_merge is done!!
换用0.25阈值的数据处理完毕，总的需要处理个数为23731，剩余未处理pw个数为8992，已处理个数为14739


In [6]:
#处理类中单个商品间的相似度
import markov_clustering as mc
import networkx as nx
import numpy as np
from  gensim import models
from scipy.sparse import *
from collections import defaultdict
def load_multy_data():
    data={}
    data_vec={}
    num_id=0
    #记录num_id对应的edges的编号，方便后面找到相连的边;以及edges对应的num_id
    num_id_edges={}
    with open('one_cluster_pw', encoding='utf8') as file:     
        for line in file:
            cons = line.strip()
            vec=vec_Glove[str(rn_num[cons])]
            data[str(num_id)]=cons
            data_vec[str(num_id)]=vec
            num_id_edges[num_id]=str(rn_num[cons])
            num_id+=1
    return data,data_vec,num_id,num_id_edges
data,data_vec,num_id,num_id_edges=load_multy_data()

expansion = 10  # 使得不同的区域之间的联系加强
inflation = 1.1  # 各个数据幂次方，强化紧密的点，弱化松散的点
loop_value = 1  # 自循环的概率
iterations = 1000
pruning_threshold = 0.005
pruning_frequency = 0.75              
class MCL(object):
    def __init__(self, id2pw: dict):
        self.id2pw = id2pw
        self.cluster_name = "one_review_cluster"
        self.rbcids = []

    def run_from_mat(self, sim_mat):
        result = mc.run_mcl(sim_mat,
                            expansion=expansion,
                            inflation=inflation,
                            loop_value=loop_value,
                            iterations=iterations,
                            pruning_threshold=pruning_threshold,
                            pruning_frequency=pruning_frequency,
                            verbose=True
                            )  # run MCL with default parameters
        rbcids = mc.get_clusters(result)  # get clusters
        
        for item in rbcids:
            sub_item = []
            for i in range(len(item)):
                pw = self.id2pw.get(str(item[i]))
                if pw is not None:
                    sub_item.append(pw)
            self.rbcids.append(sub_item)
        #print(self.rbcids)

    def save_cluster(self):
        file = open(self.cluster_name, 'w', encoding='utf8')
        file.write("#############\n")
        file.write("expansion={0}\ninflation={1}\n"
                   "loop_value={2}\niterations={3}\n"
                   "pruning_threshold={4}\n"
                   "pruning_frequency={5}\n".format(expansion, inflation,
                                                    loop_value, iterations,
                                                    pruning_threshold,
                                                    pruning_frequency))
        file.write("#############\n")
        for item in self.rbcids:
            for i in range(len(item)):
                pw = item[i]
                try:
                    #cid1=pw.split('-')[0]
                    #cid1_name=id1_name[cid1]
                    #pw=cid1_name+'-'+pw.split('-')[1]
                    file.write(pw+", " if i < len(item)-1 else pw)
                except:
                    print(pw)
            file.write("\n")
        file.close()
def graph_process():
    print(num_id)
    sim_mat = lil_matrix((num_id, num_id))
    for i in range(num_id-2):
        if i%1000==0:
            print(i)
        lst = edges.get(num_id_edges[i])
        for j in range(i+1,num_id):
            if num_id_edges[j] in lst:
                val=_func(data_vec[str(i)],data_vec[str(j)])
                try:
                    if val>0.2:
                        sim_mat[i, j] = val
                        sim_mat[j, i] = val
                except Exception as e:
                    #print(e) 
                    continue
    return sim_mat

graph_dict=graph_process()
print('相似矩阵计算完毕')
A=MCL(data)
A.run_from_mat(graph_dict)
A.save_cluster()
print('All done')              
               

11510
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
相似矩阵计算完毕
--------------------------------------------------
MCL Parameters
Expansion: 10
Inflation: 1.1
Pruning threshold: 0.005, frequency: 0.75 iteration
Convergence check: 1 iteration
Maximum iterations: 1000
Sparse matrix mode
--------------------------------------------------
Iteration 1
Checking for convergence
Iteration 2
Checking for convergence
Iteration 3
Checking for convergence
Iteration 4
Checking for convergence
Iteration 5
Checking for convergence
Iteration 6
Checking for convergence
Iteration 7
Checking for convergence
Iteration 8
Checking for convergence
Iteration 9
Checking for convergence
Iteration 10
Checking for convergence
Iteration 11
Checking for convergence
Iteration 12
Checking for convergence
Iteration 13
Checking for convergence
Iteration 14
Checking for convergence
Iteration 15
Checking for convergence
Iteration 16
Checking for convergence
Iteration 17
Checking for convergence
Iteration 18
Che

In [7]:
#处理类中多个商品间的相似度
import markov_clustering as mc
import networkx as nx
import numpy as np
from  gensim import models
from scipy.sparse import *
from collections import defaultdict
def load_multy_data():
    data={}
    data_vec={}
    num_id=0
    #记录num_id对应的edges的编号，方便后面找到相连的边;以及edges对应的num_id
    num_id_edges={}
    with open('multy_test_review.txt', encoding='utf8') as file:     
        for line in file:
            flag=[item for item in line.split(',')]
            if len(flag)==1:
                continue
            cons=[]
            vec=np.array([0.0 for i in range(300)])
            num_id_edges[num_id]=[]
            for item in line.strip().split(','):
                if item=='':
                    continue
                item=item.strip()
                cons.append(item)
                if item=='' or item=='=':
                    continue
                vec+=np.array(vec_Glove[str(rn_num[item])])
                num_id_edges[num_id]+=edges.get(str(rn_num[item]))
            vec=vec/len(cons)
            data[str(num_id)]=cons
            data_vec[str(num_id)]=vec
            num_id+=1
    return data,data_vec,num_id,num_id_edges
data,data_vec,num_id,num_id_edges=load_multy_data()

expansion = 2  # 使得不同的区域之间的联系加强
inflation = 1.1  # 各个数据幂次方，强化紧密的点，弱化松散的点
loop_value = 1  # 自循环的概率
iterations = 1000
pruning_threshold = 0.005
pruning_frequency = 0.75              
class MCL(object):
    def __init__(self, id2pw: dict):
        self.id2pw = id2pw
        self.cluster_name = "multy_test_cluster"
        self.rbcids = []

    def run_from_mat(self, sim_mat):
        result = mc.run_mcl(sim_mat,
                            expansion=expansion,
                            inflation=inflation,
                            loop_value=loop_value,
                            iterations=iterations,
                            pruning_threshold=pruning_threshold,
                            pruning_frequency=pruning_frequency,
                            verbose=False
                            )  # run MCL with default parameters
        rbcids = mc.get_clusters(result)  # get clusters
        
        for item in rbcids:
            print(item)
            sub_item = []
            for i in range(len(item)):
                pw = self.id2pw.get(str(item[i]))
                if pw is not None:
                    sub_item.append(pw)
            self.rbcids.append(sub_item)
    def save_cluster(self):
        file = open(self.cluster_name, 'w', encoding='utf8')
        file.write("#############\n")
        file.write("expansion={0}\ninflation={1}\n"
                   "loop_value={2}\niterations={3}\n"
                   "pruning_threshold={4}\n"
                   "pruning_frequency={5}\n".format(expansion, inflation,
                                                    loop_value, iterations,
                                                    pruning_threshold,
                                                    pruning_frequency))
        file.write("#############\n")
        for item in self.rbcids:
            t=[]
            out=[t.extend(i) for i in item]
            #print (out)
            for i in range(len(item)):
                pw = item[i]
                #print(pw)
                try:
                    for i in range(len(pw)):
                        file.write(pw[i]+',' if i<len(pw)-1 else pw[i])
                except:
                    print(item)
            file.write('\n')       
        file.close()
def graph_process():
    print(num_id)
    sim_mat = lil_matrix((num_id, num_id))
    for i in range(num_id-2):
        if i%1000==0:
            print(i)
        for j in range(i+1,num_id):
                val=_func(data_vec[str(i)],data_vec[str(j)])
                try:
                    if val>0.5:
                        sim_mat[i, j] = val
                        sim_mat[j, i] = val
                except Exception as e:
                    #print(e) 
                    continue
    return sim_mat
graph_dict=graph_process()
#print(graph_dict)
print('相似矩阵计算完毕')
A=MCL(data)
A.run_from_mat(graph_dict)
A.save_cluster()
print('All done')              
               
               

KeyError: '服饰内衣-腹肌'

In [6]:
#手動處理類別中的商品
#----------------------人工review前的数据------------------------------------

#读取未进入人工review阶段的单类商品,对他们进行阈值0.25调整后划分，剩下的单独成类
def Data_without_processed_by_thre():
    data_without_processed_by_thre=[]
    with open('manual_judge_word_rm_test', encoding='utf8') as file:
        for line in file:
            cons = line.strip()
            data_without_processed_by_thre.append(cons)
    return data_without_processed_by_thre

#----------------------人工review后的数据------------------------------------
#读取人工review完毕后的总数据
def load_review_data():
    #分为分好类的数据，删选出来不属于该类的商品和没有sku的商品
    data_done=[]
    data_del=[]
    data_del_cluster={}
    data_without_sku=[]
    with open('data_review_test', encoding='utf8') as file:
        for line in file:
            cons = [pw.strip() for pw in line.strip().split(",") if pw!='']
            data_done.append(cons)
    return data_done,data_del,data_without_sku
              

#按照0.3仍未被处理的词，改用0.25阈值
data_without_processed_by_thre=Data_without_processed_by_thre()
data_done,data_del,data_without_sku=load_review_data()


#----------------------定义合并数据的方法------------------------------------ 
#需要被归入的大类数据，需要合并的单类数据，输入阈值
def data_merge(base,process,thre):
    #维护长度为3的堆
    top_n = 3
    base_vec={}
    base_id_code={}
    base_id=0
    #编码所在的list序列号
    code_list={}
    for cons in base:
        vec=np.array([0.0 for i in range(300)])
        tmp_code=[]
        for item in cons:
            if item=='' or item=='=' or item=='-':
                continue
            vec+=vec_Glove[name_id(item)]
            tmp_code.append(name_id(item))
            code_list[name_id(item)]=base_id
        base_id_code[base_id]=tmp_code
        base_vec[base_id]=vec/len(cons)
        base_id+=1
    remove_pw=[]
    line=0
    for pw in process:
        if line%1000==0:
            print(line)
        line+=1
        #vec_tmp为要处理的pw的词向量
        vec_tmp = np.array(vec_Glove[name_id(pw)])
        keys = []
        vals = []
        if name_id(pw) in edges.keys():
            for edge in edges[name_id(pw)]:
                try:
                    vec_edge=vec_Glove[edge]
                    val=_func(vec_tmp,vec_edge)
                    if len(keys) < top_n:
                            keys.append(edge)
                            vals.append(val)
                    else:
                        min_val = vals[0]
                        min_idx = 0
                        for i in range(top_n):
                            if vals[i] < min_val:
                                min_val = vals[i]
                                min_idx = i
                        if val > min_val:
                            keys[min_idx] = edge
                            vals[min_idx] = val
                except:
                    continue
        key_val={}
        for i in range(len(keys)):
            key_val[keys[i]]=vals[i]
        new=sorted(key_val.items(),key=lambda x:x[1],reverse=True)
        keys,vals=[],[]
        for content in new:
            keys.append(content[0])
            vals.append(content[1])
        for i in range(top_n):
            try:
                if keys[i] in code_list.keys():
                    if vals[i]>thre:
                        base[code_list[keys[i]]].append(pw)
                        #满足单个在其他类中，即被处理的词
                        if pw not in remove_pw:
                            remove_pw.append(pw)
                        break
            except:                    
                continue
    process_left=[]
    for item in process:
        if item not in remove_pw:
            process_left.append(item)
    print('Data_merge is done!!')
    return base,process_left,remove_pw


#----------------------处理按照0.3仍未被处理的词，改用0.25阈值------------------------------------ 
data_done,data_without_processed_by_thre_left,data_without_processed_by_thre_remove_pw=data_merge(data_done,data_without_processed_by_thre,0)
print('换用0.25阈值的数据处理完毕，总的需要处理个数为{}，剩余未处理pw个数为{}，已处理个数为{}'.format(len(data_without_processed_by_thre),len(data_without_processed_by_thre_left),len(data_without_processed_by_thre_remove_pw)))

print(data_done)
print('----------------------')
print(data_without_processed_by_thre_left)



0
Data_merge is done!!
换用0.25阈值的数据处理完毕，总的需要处理个数为191，剩余未处理pw个数为11，已处理个数为180
[['珠宝首饰-项链', '珠宝首饰-吊坠', '珠宝首饰-锁骨链', '珠宝首饰-黄金吊坠', '珠宝首饰-银项链', '珠宝首饰-挂坠', '珠宝首饰-颈链', '珠宝首饰-项坠', '珠宝首饰-金吊坠', '珠宝首饰-金项链', '珠宝首饰-纯银项链', '珠宝首饰-佛吊坠', '珠宝首饰-观音', '珠宝首饰-项圈', '珠宝首饰-护身符', '珠宝首饰-颈带', '珠宝首饰-银吊坠', '珠宝首饰-长项链', '珠宝首饰-金坠子', '珠宝首饰-足银项链', '珠宝首饰-颈圈', '珠宝首饰-短项链', '珠宝首饰-银项圈', '珠宝首饰-脖圈', '珠宝首饰-毛衣链', '珠宝首饰-挂链', '珠宝首饰-项链饰品', '珠宝首饰-脖链', '珠宝首饰-单链子', '二手商品-长项链', '二手商品-银项链', '二手商品-挂链', '二手商品-黄金吊坠', '二手商品-金吊坠', '二手商品-颈链', '珠宝首饰-首饰饰品', '珠宝首饰-脖颈链', '珠宝首饰-算盘', '珠宝首饰-项链坠子', '珠宝首饰-小吊坠', '珠宝首饰-链坠', '珠宝首饰-麋鹿角', '珠宝首饰-银牌', '珠宝首饰-骨链', '珠宝首饰-衣链', '珠宝首饰-吊牌', '珠宝首饰-大吊坠', '珠宝首饰-银坠子', '珠宝首饰-阴阳鱼', '珠宝首饰-佛坠', '珠宝首饰-金项链坠', '珠宝首饰-许愿石', '珠宝首饰-萌鸡', '珠宝首饰-蜜豆', '珠宝首饰-猫项链', '珠宝首饰-银坠', '珠宝首饰-转运石', '珠宝首饰-颈饰品', '珠宝首饰-权杖', '珠宝首饰-福字贴', '珠宝首饰-名字牌', '珠宝首饰-灵符', '珠宝首饰-手工皂', '珠宝首饰-玉树', '珠宝首饰-汉白玉', '珠宝首饰-楼梯', '珠宝首饰-鸡尾酒', '珠宝首饰-白骨精', '珠宝首饰-舞裙', '珠宝首饰-翡翠兔', '珠宝首饰-摆钟', '珠宝首饰-花籽', '珠宝首饰-钥匙环扣', '珠宝首饰-相机', '珠宝首饰-关公刀', '珠宝首饰-花板', '家居日用-护甲', '珠宝首饰-浮萍', '珠宝首饰-机油', '珠